In [1]:
import os, re
import pandas as pd
import numpy as np
from konlpy.tag import Okt
okt=Okt()

In [2]:
#university 만 수정
university='hufs'

path=f'/data/yogiyo/pr_data/{university}/'
shop1=pd.read_csv(path+f'{university}_shop1.csv')
shop2=pd.read_csv(path+f'{university}_shop2.csv')
shop3=pd.read_csv(path+f'{university}_shop3.csv')


In [3]:
shop=pd.concat([shop1,shop2,shop3])

In [4]:
del shop['Unnamed: 0']

In [5]:
shop.shape

(1338, 4)

In [6]:
def find_in(x):
    if x !=x:
        return False
    find=['리뷰작성','리뷰 작성','리뷰 이벤트','리뷰이벤트','후기이벤트','후기 이벤트','리뷰','서비스','써비스','참여','증정','약속']
    cnt=0
    for tok in find:
        if re.search(tok, x):
            cnt+=1
        if cnt ==2:
            return True
    return False

In [7]:
def review_line(notice,isservice):
    if isservice==False:
        return None
    find=['리뷰','이벤트','요청','사항','서비스','참여','찜','별 점','별점','꼭',
      '제공','선택','약속','작성','결제','닉네임','사진',
     '써비스','랜덤','후기','써비스','증정','안심','번호','포토','꾹']
    
    notice_split=notice.split('\n')
    extract=''
    for sent in notice_split:
        for word in find:
            if re.search(word,sent):
                extract=extract+'\n'+sent
                break
    return extract

In [8]:
shop['isservice']=shop['notice'].map(lambda x: find_in(x))
shop['reviewevent']=shop.apply(lambda x: review_line(x.notice,x.isservice),axis=1)

In [9]:
def extract_photo(sent):
    if sent == None :
        return None
    extract=None
    sent_split=sent.split('\n')
    find=['사진','포토']
    for sent in sent_split:
        for word in find:
            if re.search(word,sent):
                if extract==None:
                    extract=sent
                else:
                    extract=extract+'\n'+sent
                break
    return extract

In [10]:
def extract_star(sent):
    if sent == None :
        return None
    
    
    extract=None
    sent_split=sent.split('\n')
    find=['별']
    for sent in sent_split:
        token=okt.morphs(sent)
        for word in find:
            if word in token:
                if extract==None:
                    extract=sent
                else:
                    extract=extract+'\n'+sent
                break
    return extract

In [11]:
shop['photo']=shop.reviewevent.map(lambda x: extract_photo(x))

In [12]:
shop['star5']=shop.reviewevent.map(lambda x: extract_star(x))

In [13]:
def make_catagory(isservice,photo,star5):
    if not isservice:
        return 4
    cnt=0
    if photo != None:
        cnt+=1
    if star5 != None:
        cnt+=2
    return cnt

In [14]:
shop['catagory']=shop.apply(lambda x: make_catagory(x.isservice,x.photo,x.star5),axis=1)

In [15]:
pd.set_option('display.max_row', None)
shop

,shop,star,review_cnt,notice,isservice,reviewevent,photo,star5,catagory
0,김밥&즉석반찬전문점소담찬,5.0,3,NaN,False,None,None,None,4
1,대식가2900,5.0,1,모든 고기는 숯불에 구워서 나갑니다~\n삼겹살은 48시간 저온숙성된 숙성삼겹살 입니...,True,\n다른 고기들은 이베리코 입니다. 찜.리뷰이벤트 메모 남기시면 계란찜 서비스 드립니다,None,None,0
2,맛집옛날통닭-경희대점,4.8,40,NaN,False,None,None,None,4
3,불꽃냉면칼국수,4.7,137,NaN,False,None,None,None,4
4,탐앤탐스-상봉메가박스점,4.8,84,♡♡매장을 찜해두시면 더욱 간편하게 주문 가능합니다♡♡\n탐앤탐스입니다.\n프레즐과...,False,None,None,None,4
5,계절이야기,4.4,210,★★고객님의 소중한 리뷰를 빵빵! 남겨주시면 '계란찜'을 서비스로 드려요~~~★★\...,True,\n★★고객님의 소중한 리뷰를 빵빵! 남겨주시면 '계란찜'을 서비스로 드려요~~~★★,None,None,0
6,배스킨라빈스-월곡역점,4.8,342,NaN,False,None,None,None,4
7,북촌손만두-경희대점,4.0,7,NaN,False,None,None,None,4
8,왕건고기촌육회집,4.9,16,안녕하세요\n왕건고기촌입니다.^^\n저희는 중화역 근처에서 정육식당을 10년 이상 ...,False,None,None,None,4
9,라마파스타,4.7,3,NaN,False,None,None,None,4


In [16]:
shop.to_csv(path+f'{university}_shop.csv',index=False)